In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import logging

module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from pvi.models.sgp import SparseGaussianProcessModel
from pvi.utils.gaussian import mvstandard2natural, mvnatural2standard

import torch
import numpy as np
import matplotlib.pyplot as plt
import gpytorch
import tqdm.auto as tqdm

from torch import nn
from gpytorch.kernels import ScaleKernel, RBFKernel

%matplotlib inline
torch.set_default_dtype(torch.float64)

# Set up data and helper functions

In [ ]:
x = np.linspace(-1, 1, 50)
y = 2 * np.sin(5*x) + 3 * np.abs(x) * np.random.rand(len(x))

x = torch.tensor(x).unsqueeze(1)
y = torch.tensor(y).unsqueeze(1)

In [ ]:
def plot_data(x, y):
    plt.figure()
    plt.grid(b=True)
    plt.scatter(x, y)
    plt.show()
    
def plot_results(x, y, model, q):
    pp = model(x, q)
    mean = pp.mean.detach()
    std = pp.variance.detach() ** 0.5
    
    samples = pp.sample((20,))
    
    plt.figure()
    plt.grid(b=True)
    
    for sample in samples:
        plt.plot(x, sample, color='k', alpha=.1)
        
    plt.plot(x.squeeze(-1), mean)
    plt.fill_between(x.squeeze(-1), mean-1.96*std, mean+1.96*std, alpha=.25)
    plt.scatter(x, y)
    plt.show()
    
def plot_training(training_array):
    x_vals = np.arange(1, len(training_array)+1)
    plt.figure()
    plt.grid(b=True)
    plt.plot(x_vals, training_array)
    plt.ylabel('ELBO Loss')
    plt.xlabel('Step')
    plt.show()
    
data = {
    "x": x,
    "y": y,
}

In [ ]:
plot_data(x, y)

# Construct SGP model

In [ ]:
hyperparameters = {
    "D": 1,
    "num_inducing": 5,
    "kernel_class": RBFKernel,
    "kernel_params": {"lengthscale": .5},
    "epochs": 500,
    "optimiser_params": {"lr": 1e-3},
    "batch_size": 50
}

inducing_locations = x[::10].clone()

model = SparseGaussianProcessModel(inducing_locations=inducing_locations, 
                                   output_sigma=1., hyperparameters=hyperparameters)

q = {
    "distribution": torch.distributions.MultivariateNormal(
        torch.zeros(hyperparameters["num_inducing"]), torch.eye(hyperparameters["num_inducing"])),
    "nat_params": {
        "np1": torch.zeros(hyperparameters["num_inducing"]),
        "np2": -0.5 * torch.eye(hyperparameters["num_inducing"])
    }
}

In [ ]:
plot_results(x, y, model, q)

# Fit data

In [ ]:
t = {
    "nat_params": {
        "np1": torch.tensor([0.]*model.hyperparameters["num_inducing"]),
        "np2": torch.tensor([0.]*model.hyperparameters["num_inducing"]).diag_embed()
    }
}

## First without optimising inducing locations

In [ ]:
q_new, t_new = model.conjugate_update(data, q, t)

In [ ]:
qmu, qcov = mvnatural2standard(q_new["nat_params"]["np1"], q_new["nat_params"]["np2"])
q_dist = torch.distributions.MultivariateNormal(qmu, covariance_matrix=qcov)

q_new["distribution"] = q_dist

In [ ]:
plot_results(x, y, model, q_new)

## Now optimising inducing locations and kernel hyperparameters

In [ ]:
def fit(model, data, q, t_i):
    # Set up optimiser.
    if model.hyperparameters["optimiser_class"] is not None:
        optimiser = model.hyperparameters["optimiser_class"](
            model.parameters(), **model.hyperparameters["optimiser_params"]
        )
    else:
        optimiser = optim.Adam(
            model.parameters(), **model.hyperparameters["optimiser_params"]
        )
        
    # Local optimisation to find new parameters.
    training_curves = {
        "elbo": [],
    }
    
    # Compute local factor and current global posterior.
#     with torch.no_grad():
    q, t_i = model.conjugate_update(data, q, t_i)

    q_mu, q_cov = mvnatural2standard(q["nat_params"]["np1"], q["nat_params"]["np2"])
    q_dist = torch.distributions.MultivariateNormal(q_mu, covariance_matrix=q_cov)
    q["distribution"] = q_dist
    
    epoch_iter = tqdm.tqdm(range(model.hyperparameters["epochs"]), desc="Epochs")
    for i in epoch_iter:
        # Compute cavity distribution.
        qcav = {
            "nat_params": {
                "np1": q["nat_params"]["np1"] - t_i["nat_params"]["np1"],
                "np2": q["nat_params"]["np2"] - t_i["nat_params"]["np2"],
            }
        }
        qcav_mu, qcav_cov = mvnatural2standard(qcav["nat_params"]["np1"], qcav["nat_params"]["np2"])
        qcav_dist = torch.distributions.MultivariateNormal(qcav_mu, qcav_cov)
        qcav["distribution"] = qcav_dist
        
        # Compute ELBO.
        elbo = model.elbo(data, q, qcav)
        loss = -elbo
        
        # Backwards step.
        loss.backward()
        optimiser.step()
        optimiser.zero_grad()
        
        # Compute local factor and current global posterior.
#         with torch.no_grad():
        q, t_i = model.conjugate_update(data, q, t_i)

        q_mu, q_cov = mvnatural2standard(q["nat_params"]["np1"], q["nat_params"]["np2"])
        q_dist = torch.distributions.MultivariateNormal(q_mu, covariance_matrix=q_cov)
        q["distribution"] = q_dist
        
        training_curves["elbo"].append(elbo.item())
        
    return q, t_i, training_curves

In [ ]:
q_new, t_new, training_curves = fit(model, data, q, t)

In [ ]:
plot_training(training_curves["elbo"])

In [ ]:
plot_results(x, y, model, q_new)